<a href="https://colab.research.google.com/github/williamwei2/UserBehavior/blob/master/Taobao_User_Behavior_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**淘宝用户数据推荐系统研究** Recommendation System
<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/Recommendation%20System/Recommendation%20System.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/Recommendation%20System/Recommendation%20System.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

A recommendation system seeks to predict the rating or preference a user would give to an item given his old item ratings or preferences. Recommendation systems are used by pretty much every major company in order to enhance the quality of their services.

In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


## Loading in data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/goodbooks-10k/ratings.csv",sep=',')

In [0]:
#dataset = pd.read_csv("https://raw.githubusercontent.com/TannerGilbert/Tutorials/master/Recommendation%20System/ratings.csv",thousands=',')

读取UserBehavior 数据（亿级）

In [0]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/TAOBAO_user_behavior_dataset/UserBehavior.csv",sep=',',header=None,names=['userid','itemid','categoryid','behavior','timestamp'])

In [0]:
#dataset.drop('categoryid',axis=1,inplace=True)

In [0]:
#dataset.drop('timestamp',axis=1,inplace=True)

In [10]:
dataset.head()

,userid,itemid,categoryid,behavior,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [0]:
dataset['behavior'].replace({'pv':1,'cart':2,'fav':3,'buy':4},inplace=True)

In [0]:
dataset.drop(dataset[(dataset.userid>100)&(dataset.itemid>100)].index, inplace=True)

In [13]:
dataset.head()

,userid,itemid,categoryid,behavior,timestamp
0,1,2268318,2520377,1,1511544070
1,1,2333346,2520771,1,1511561733
2,1,2576651,149192,1,1511572885
3,1,3830808,4181361,1,1511593493
4,1,4365585,2520377,1,1511596146


In [14]:
dataset.shape

(10947, 5)

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [16]:
train.head()

,userid,itemid,categoryid,behavior,timestamp
84665219,29,569369,1879194,1,1512307921
97052921,86,4419569,1528701,1,1512178159
50878223,58,802546,4882154,1,1512112435
87990232,44,4808632,2217187,1,1511681260
94260883,73,3709994,2096639,1,1511670506


In [17]:
test.head()

,userid,itemid,categoryid,behavior,timestamp
17770020,90,3639754,2247787,1,1511971870
91711409,61,312793,4882154,2,1512295579
98262182,914544,81,4756105,1,1511692342
38827059,95,941139,965809,1,1511961161
26900784,4,3019619,4801426,1,1511946525


In [18]:
n_users = len(dataset.userid.unique())
n_users

1160

In [19]:
n_items = len(dataset.itemid.unique())
n_items

7106

In [0]:
import tensorflow as tf

In [0]:
vocabsize=10000000

## Creating dot product model
Most recommendation systems are build using a simple dot product as shown below but newer ones are now implementing a neural network instead of the simple dot product.

In [0]:
# creating book embedding path
#item_input = tf.keras.Input(shape=[1],name="Item-Input")
#item_embedding = tf.keras.layers.Embedding(vocabsize+1,4, name="Item-Embedding")(item_input)
#item_vec = tf.keras.layers.Flatten(name="Flatten-Items")(item_embedding)

# creating user embedding path
#user_input = tf.keras.Input(shape=[1],name="User-Input")
#user_embedding = tf.keras.layers.Embedding(vocabsize+1,4, name="User-Embedding")(user_input)
#user_vec = tf.keras.layers.Flatten(name="Flatten-Users")(user_embedding)

# performing dot product and creating model
#prod = tf.keras.layers.Dot(axes=1)([item_vec, user_vec])
#model = tf.keras.Model([user_input, item_input], prod)
#model.compile('adam', 'mean_squared_error')

In [0]:
#from keras.models import load_model

#if os.path.exists('/content/drive/My Drive/Colab Notebooks/Models/dotProductModel.h5'):
    #model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Models/dotProductModel.h5')
#else:
   # history = model.fit([train.userid, train.itemid], train.behavior, epochs=5, verbose=1)
   # model.save('/content/drive/My Drive/Colab Notebooks/Models/dotProductModel.h5')
   # plt.plot(history.history['loss'])
   # plt.xlabel("Epochs")
   # plt.ylabel("Training Error")

In [0]:
#model.evaluate([test.userid, test.itemid], test.behavior)

In [0]:
#predictions = model.predict([test.userid.head(10), test.itemid.head(10)])

#[print(predictions[i], test.behavior.iloc[i]) for i in range(0,10)]

## Creating Neural Network
Neural Networks proved there effectivness for almost every machine learning problem as of now and they also perform exceptionally well for recommendation systems.

In [0]:
# creating book embedding path
item_input = tf.keras.Input(shape=[1], name="Item-Input")
item_embedding = tf.keras.layers.Embedding(vocabsize+1, 5, name="Item-Embedding")(item_input)
item_vec = tf.keras.layers.Flatten(name="Flatten-Items")(item_embedding)

# creating user embedding path
user_input = tf.keras.Input(shape=[1], name="User-Input")
user_embedding = tf.keras.layers.Embedding(vocabsize+1, 5, name="User-Embedding")(user_input)
user_vec = tf.keras.layers.Flatten(name="Flatten-Users")(user_embedding)

# concatenate features
conc = tf.keras.layers.Concatenate()([item_vec, user_vec])

# add fully-connected-layers
fc1 = tf.keras.layers.Dense(128, activation='relu')(conc)
fc2 = tf.keras.layers.Dense(32, activation='relu')(fc1)
out = tf.keras.layers.Dense(1)(fc2)

# Create model and compile it
model2 = tf.keras.Model([user_input, item_input], out)
model2.compile('adam', 'mean_squared_error')

In [0]:
from keras.models import load_model

if os.path.exists('/content/drive/My Drive/Colab Notebooks/Models/NeuralNetworkModel_withName_inLayer.h5'):
    model2 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Models/NeuralNetworkModel_withName_inLayer.h5',compile=True)
else:
    history = model2.fit([train.userid, train.itemid], train.behavior, epochs=5, verbose=1)
    model2.save('/content/drive/My Drive/Colab Notebooks/Models/NeuralNetworkModel_withName_inLayer.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [28]:
model2.evaluate([test.userid, test.itemid], test.behavior)

2190/2190 [==============================] - 0s 175us/sample - loss: 0.5117


0.5117225947445386

In [29]:
predictions = model2.predict([test.userid.head(10), test.itemid.head(10)])

[print(predictions[i], test.behavior.iloc[i]) for i in range(0,10)]

[1.0704815] 1
[1.8095574] 2
[1.0965444] 1
[1.6173295] 1
[1.0293572] 1
[1.3543851] 1
[1.950447] 3
[1.1872991] 1
[1.8023701] 1
[1.0309834] 1


[None, None, None, None, None, None, None, None, None, None]

## Visualizing Embeddings
Embeddings are weights that are learned to represent some specific variable like books and user in our case and therefore we can not only use them to get good results on our problem but also to extract inside about our data.

In [0]:
# Extract embeddings
#item_em = model2.get_layer('Item-Embedding')
#item_em_weights = item_em.get_weights()[0]

In [0]:
#item_em_weights[:5]

In [0]:
#from sklearn.manifold import TSNE

#tsne = TSNE(n_components=2, verbose=1, perplexity=4, n_iter=250)
#tnse_results = tsne.fit_transform(item_em_weights)

In [0]:
#from sklearn.decomposition import PCA
#import seaborn as sns

#pca = PCA(n_components=2)
#pca_result = pca.fit_transform(item_em_weights)
#sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1])

In [0]:
#item_em_weights = item_em_weights / np.linalg.norm(item_em_weights, axis = 1).reshape((-1, 1))
#item_em_weights[0][:10]
#np.sum(np.square(item_em_weights[0]))

In [0]:
#pca = PCA(n_components=2)
#pca_result = pca.fit_transform(item_em_weights)
#sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1])

In [0]:
#sns.scatterplot(x=tnse_results[:,0], y=tnse_results[:,1])

## Making Recommendations

In [37]:
# Creating dataset for making recommendations for the first user
item_data = np.array(list(set(dataset.itemid)))
item_data[:5]

array([1, 3, 4, 5, 6])

In [38]:
user = np.array([1 for i in range(len(item_data))])
user[:5]

array([1, 1, 1, 1, 1])

In [39]:
predictions = model2.predict([user, item_data])

predictions = np.array([a[0] for a in predictions])

recommended_item_ids = (-predictions).argsort()[:5]

recommended_item_ids

array([1809, 6097, 1312, 4272, 4089])

In [40]:
# print predicted scores
predictions[recommended_item_ids]

array([2.8701148, 2.6623483, 2.5920224, 2.5450172, 2.5194702],
      dtype=float32)

In [49]:
items = pd.read_csv("/content/drive/My Drive/Colab Notebooks/TAOBAO_user_behavior_dataset/UserBehavior.csv",sep=',',header=None,names=['userid','itemid','categoryid','behavior','timestamp'])
items.head()

,userid,itemid,categoryid,behavior,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [0]:
items.drop('categoryid',axis=1,inplace=True)
items.drop('userid',axis=1,inplace=True)
items.drop('behavior',axis=1,inplace=True)
items.drop('timestamp',axis=1,inplace=True)

In [0]:
items.drop(items[(items.itemid<100)].index, inplace=True)

In [53]:
items.head()

,itemid
0,2268318
1,2333346
2,2576651
3,3830808
4,4365585


In [55]:
items.tail()

,itemid
100150802,4797808
100150803,4613472
100150804,3647364
100150805,1903801
100150806,3696094


In [54]:
items[items['itemid'].isin(recommended_item_ids)]

,itemid
4441882,4272
6677194,4272
6677196,4272
7992980,4272
14029076,4272
...,...
88906115,1312
92082410,1312
92564700,6097
94392752,1312
